THIS IS THE SECOND PART OF THE PROJECT. HERE WE WILL FIT THE THREE SELECTED MODELS ON THE ENTIRE DATASET WITHOUT
ELIMINATING ANY FEATURES AND COMPARE THE OUTCOMES TO STEP THREE WHICH EMPLOYS ONLY PART OF THE FEATURES. 

WE WILL VARY THE AMOUNT OF DATA USED IN TESTING THE MODEL AND SEE HOW IT AFFECTS THE PERFORMANCE OF THE MODELS
WE WILL USE 30% TRAINING DATA IN THE FIRST CYCLE AND USE 20% TRAINING DATA IN THE SECOND CYCLE.

RESULTS FOR EACH CYCLE ARE SHOWN IN THE LAST CELL

In [19]:
#import requisite libraries and packages for the project
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
#load the final clean, imputed and encoded dataset from the previous process
df_1 = pd.read_csv('../datasets/diabetes_dropped_imputed_encoded.csv')

exploring the dataset to ensure it was not corrupted 


In [6]:
#exploring the dataset to ensure it was not corrupted 
df_1

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,...,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn,height_meters,BMI,hip_waist_ratio,diabetic
0,0,203.0,27,56.0,3.6,4.310000,0,27,0,62.0,...,59.0,152.382979,92.524823,29.0,38.0,720.0,1.5748,48.790420,0.763158,0
1,1,165.0,42,24.0,6.9,4.440000,0,10,0,64.0,...,68.0,152.382979,92.524823,46.0,48.0,360.0,1.6256,82.495282,0.958333,0
2,2,228.0,37,37.0,6.2,4.640000,0,39,0,61.0,...,92.0,185.000000,92.000000,49.0,57.0,180.0,1.5494,106.638214,0.859649,0
3,3,78.0,38,12.0,6.5,4.630000,0,48,1,67.0,...,50.0,152.382979,92.524823,33.0,38.0,480.0,1.7018,41.089412,0.868421,0
4,4,249.0,35,28.0,8.9,7.720000,0,45,1,68.0,...,80.0,152.382979,92.524823,44.0,41.0,300.0,1.7272,61.343116,1.073171,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,398,296.0,113,46.0,6.4,16.110001,1,34,1,69.0,...,94.0,130.000000,94.000000,35.0,39.0,210.0,1.7526,56.322314,0.897436,1
399,399,284.0,34,54.0,5.3,4.390000,1,32,0,63.0,...,100.0,146.000000,102.000000,32.0,43.0,180.0,1.6002,60.141214,0.744186,0
400,400,194.0,105,38.0,5.1,13.630000,1,10,0,69.0,...,70.0,152.382979,92.524823,33.0,40.0,20.0,1.7526,54.368939,0.825000,1
401,401,199.0,21,52.0,3.8,4.490000,1,22,0,63.0,...,78.0,152.382979,92.524823,41.0,48.0,255.0,1.6002,76.933890,0.854167,0


In [7]:
df_1.shape

(403, 23)

In [8]:
df_1.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio', 'diabetic'],
      dtype='object')

In [9]:
df_1.isnull().values.any()

False

In [10]:
df_1.isnull().sum().sum()

0

In [14]:
df_1.columns

Index(['id', 'chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'location', 'age',
       'gender', 'height', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio', 'diabetic'],
      dtype='object')

In [15]:
#selecting the columns to be used for training the model
#selecting the feature and the predictors of the model.
#diabetic - a feature created in Part I is set as the y variable 
#all the other features will be used to predict y
train_cols = ['chol', 'stab.glu', 'hdl', 'ratio', 'glyhb', 'age',
       'gender', 'weight', 'frame', 'bp.1s', 'bp.1d', 'bp.2s',
       'bp.2d', 'waist', 'hip', 'time.ppn', 'height_meters', 'BMI',
       'hip_waist_ratio']

x = df_1[train_cols]
y = df_1['diabetic']

In [116]:
#splitting the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=17)

In [117]:
#scaling the data ensures faster training and improved scoring for machine learning models
x_train_scaled =  StandardScaler().fit_transform(x_train)
x_test_scaled = StandardScaler().fit_transform(x_test)

In [118]:
#this function fits the three moels; RF, SVM and XGB
def auto_train(x, y, model_alg):
    if model_alg == 'rf':
        model = RandomForestClassifier(random_state=30, n_estimators=100, max_features=8, max_depth=20)
    elif model_alg == 'svm':
        model = SVC(C=1, kernel='rbf', random_state=1)
    elif model_alg == 'xgb':
        model = XGBClassifier(use_label_encoder=False, objective='reg:logistic', colsample_bytree=0.8)
        
    model.fit(x_train_scaled, y_train)
    
    return model

In [119]:
#this function is for evaluating the individual models fitted in the above function
#evaluation objects are created in this function - mae, rmse, rac and clas
def auto_evaluate(model, x, y):
    
    predicted = model.predict(x)
    
    mae = mean_absolute_error(y, predicted)
    rmse = np.sqrt(mean_squared_error(y, predicted))
    rac = roc_auc_score(y, predicted)
    clas = classification_report(y, predicted)
    
    return [mae, rmse, rac, clas]

In [120]:
#calling the auto_train function and passing the positional arguments to train the model
random_forest = auto_train(x_train_scaled, y_train, 'rf')
svm = auto_train(x_train_scaled, y_train, 'svm')
gradient_boost = auto_train(x_train_scaled, y_train, 'xgb')

In [121]:
#evluatiing the SVM model
svm_output = auto_evaluate(svm, x_test_scaled, y_test)
svm_output

[0.09090909090909091,
 0.30151134457776363,
 0.7708333333333333,
 '              precision    recall  f1-score   support\n\n           0       0.90      1.00      0.95        97\n           1       1.00      0.54      0.70        24\n\n    accuracy                           0.91       121\n   macro avg       0.95      0.77      0.82       121\nweighted avg       0.92      0.91      0.90       121\n']

In [122]:
#evluatiing the RF model
random_forest_output = auto_evaluate(random_forest, x_test_scaled, y_test)
random_forest_output

[0.049586776859504134,
 0.22268088570756164,
 0.875,
 '              precision    recall  f1-score   support\n\n           0       0.94      1.00      0.97        97\n           1       1.00      0.75      0.86        24\n\n    accuracy                           0.95       121\n   macro avg       0.97      0.88      0.91       121\nweighted avg       0.95      0.95      0.95       121\n']

In [123]:
#evluatiing the XGB model
xgb_output = auto_evaluate(gradient_boost, x_test_scaled, y_test)
xgb_output

[0.049586776859504134,
 0.22268088570756164,
 0.875,
 '              precision    recall  f1-score   support\n\n           0       0.94      1.00      0.97        97\n           1       1.00      0.75      0.86        24\n\n    accuracy                           0.95       121\n   macro avg       0.97      0.88      0.91       121\nweighted avg       0.95      0.95      0.95       121\n']

THE RANDOM FOREST MODEL AND XTREME GRADIENT BOOSTER SCORE ALMOST ALL SIMILAR SCORE - THEY BOTH OUTPERFORM THE SVM MODEL BY A FEW POINTS ON ALL THE METRICS EMPLOYED ON TESTING THE MODEL.

RANDOM FORESTS ARE AN ENSEMBLE MODEL THAT COMPRISES OF MULTIPLE TREE CLASSIFIERS THAT CLASSIFY THE FEATURE USING A VOTE ON THE MODEL THAT HAS THE MOST SCORES - THIS IS A MORE OPTIMAL SCORING METHOD COMPARED TO THE SVM MODEL THAT USES HYPERPLANES TO CREATE DISTINCTIONS BETWEEN DIFFERENT CLASSES. 

THE XGB MODEL ADVANCES ON THE RANDOM FOREST TECHNIQUE IN THAT IT USES WEIGHTED MEANS FOR EACH SUB-CLASSIFIER TO EVALUATE A GENERAL SCORING USING THE HIGHEST MEAN, HENCE IT IS MORE REFINED AND ACCURATE COMPARED TO RF CLASSIFIER. IT AS WELL HAS A SHORTER TRAINING TIME COMPARED TO THE RANDOM FOREST CLASSIFER. IN THIS CASE HOWEVER, BOTH MODELS HAVE THE SAME SCORE.

USING A SMALLER TESTING DATASET (20%) DID SHOW HIGHER PERFORMANCE FOR ALL THE THREE CLASSIFIER - THIS CAN BE EXPLAINED BASED ON THE SIZE OF THE ENTIRE DATASET, WHICH IS RELATIVELY SMALL, HENCE USING MORE DATA TO TRAIN THE MODELS, IS LIKELY TO RESULT IN  BETTER SCORING ON THE TEST DATA. 

TO FIND A MORE BEFITTING MODEL, FURTHER FEATURE ENGINEERING AND INDIVIDUAL HYPERPARAMETER TUNING ARE POSSIBLE RECCOMMENDATIONS FROM THIS PROJECT. 

SINCE IT IS A MEDICAL DATA PROBLEM, IT IS ALSO IMPORTANT TO PAY MORE ATTENTION ON THE SENSITIVITY AND SPECIFICITY OF THE MODEL'S GENERALISATION TREND. 

GENERALLY, THE RF AND XGB MODELS DID PERFORM VERY WELL AS SEEN FROM THE HIGH ACCURACIES AND VERY LOW RMSE SCORE. 



sample = 0.3 (30% testing data) <>=======================<>

ROOT MEAN SQUARED EVALUATION METRIC

[Rmse (rf) = 0.222, rmse (xgb) = 0.222, rmse (svm) = 0.301]

ROC-AUC EVALUATION METRIC

[Roc-auc(rf) = 0.875, roc-auc(xgb)=0.875, roc-auc (svm) = 0.770]

CLASSIFICATION ACCURACY EVALUATION METRIC [RECALL, PRECISION, F1 SCORE]

[Accuracy (rf) = 95%, Accuracy (xgb) = 95%, Accuracy(svm) = 91%]

Recall (TP/TP+FN) ===>  SVM = , RF = , XGB =
Precision (TP/TP+FP) ===> SVM = , RF = , XGB =

sample = 0.2 (20% testing data)  <>========================<>

ROOT MEAN SQUARED EVALUATION METRIC

[Rmse (rf) = 0.222, rmse (xgb) = 0.222, rmse (svm) = 0.301]

ROC-AUC EVALUATION METRIC

[Roc-auc(rf) = 0.875, roc-auc(xgb)=0.882, roc-auc (svm) = 0.735]

CLASSIFICATION ACCURACY EVALUATION METRIC [RECALL, PRECISION, F1 SCORE]

[Accuracy (rf) = 95%, Accuracy (xgb) = 95%, Accuracy(svm) = 89%]

Recall (TP/TP+FN) ===>  SVM = , RF = , XGB =
Precision (TP/TP+FP) ===> SVM = , RF = , XGB =
